In [ ]:
!pip install tqdm
!pip install datatable

In [ ]:
from pathlib import Path 
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import datatable as dt

In [ ]:
# %%time
# train_df_og = dt.fread("data/train.jsonl",header=False)

In [ ]:
data_path = Path('data/')

In [ ]:
train_df_og = pd.read_json(data_path / 'train.jsonl', lines=True,chunksize=500000)

In [ ]:
## For pandas jsonreader object

train_dfs = []
for e, row in enumerate(tqdm(train_df_og)):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    for session, events in zip(row['session'].tolist(), row['events'].tolist()):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            event_dict['ts'].append(event['ts'])
            event_dict['type'].append(event['type'])
    
    dt.Frame(event_dict).to_jay(f"data/train_temp/{e}.jay")


In [ ]:
train_dt = dt.rbind(dt.iread("data/train_temp/*.jay"))

In [ ]:
train_dt.to_jay("data/train.jay")

In [ ]:
%%time
dt.fread("data/train.jay")

In [ ]:
## For pandas data frame read in 

%%time
train_dfs = []
for e, row in tqdm(train_df_og.iterrows()):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    for session, events in zip([row['session']], [row['events']]):
        for event in events:
            event_dict['session'].append(session)
            event_dict['aid'].append(event['aid'])
            event_dict['ts'].append(event['ts'])
            event_dict['type'].append(event['type'])

    train_dfs.append(pd.DataFrame(event_dict))
    if len(train_dfs) == 100000:
        train_dfs = pd.concat(train_dfs)
        temp_f = dt.Frame(train_dfs)
        temp_f.to_jay(f"data/train_temp/{e}.jay")
        temp_f = None
        train_dfs = []

# test_df = pd.concat(test_dfs)